<script type="text/javascript" id="MathJax-script" async
  src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
</script>

In the United States, capital gains are taxed at a flat rate, usually 20% for high earners. For example, if a high earner invests \\$100 into stocks and they sell those stocks when the stocks are worth \\$300, they would owe 20% of \\$200, which is \\$20, in taxes. 

One of the issues with this scheme is that is discourages ever selling assets to convert into other assets. Paying the capital gains tax early is financially sub-optimal because it prevents compounding. As a simple example, let's say an investor with a 20% tax rate invests 100 dollars into an asset which doubles in value. Then they can either keep it in the same asset and have it double in value again, or switch to a different asset and have that double in value. One would hope that the investor would pay the same in taxes either way, but they do not. 

In the first case, they grow \\$100 into \\$400. Then they pay 20% of \\$300 in taxes, which is \\$60. So they end up with \\$340. 

In the second case, they first grow \\$100 into \\$\200$. After capital gains tax, that becomes \\$180. Then they double it to \\$360 and pay the capital gains tax on the \\$180 increase, which is \\$36. So they end up with \\$324

This phenomenon is not isolated to this contrived example. Play with the widget below to try different scenarios; the intermediate liquidiation is always discouraged


In [34]:
import ipywidgets as widgets
from IPython.display import display

# Create widgets for the inputs
principal = widgets.FloatText(
    value=100
    ,description='Principal:'
    ,step=1
    ,style={'description_width': '200px'}
  
)

rate1 = widgets.FloatText(
    value=100
    ,description='First rate of increase (%):'
    ,step=1
    ,style={'description_width': '200px'}
 


)

rate2 = widgets.FloatText(
    value=100
    ,description='Second rate of increase (%):'
    ,step=1
    ,style={'description_width': '200px'}

)

tax_rate = widgets.FloatSlider(
    value=0.2,
    min=0.0,
    max=1.0,
    step=0.01,
    description='Tax Rate:',
    readout_format='.0%'
)

output = widgets.Output()

def compute_result(change=None):
    with output:
        output.clear_output()
        r1 = rate1.value/100 + 1
        r2 = rate2.value/100 + 1

        if r1 <0 or r2 < 0:
            print('Losses are a different thing than gains. For this widget, keep the rate of increases positive')
        else:
            # Placeholder computation
            one_liquidation = r1*r2 - tax_rate.value * (r1*r2 - 1)
            
            intmd = r1 - tax_rate.value * (r1 - 1)
            two_liquidations = intmd * r2 - tax_rate.value * (intmd * r2 - intmd)

            print(f"Intermediate value after liquidation : {principal.value * intmd:.2f}")
            print(f"Ending total, after second liquidation : {principal.value * two_liquidations:.2f}")
            print("")
            print(f"Ending total, one liquidation without intermediate: {principal.value * one_liquidation:.2f}")

# Link widget changes to update function
principal.observe(compute_result, names='value')
rate1.observe(compute_result, names='value')
rate2.observe(compute_result, names='value')
tax_rate.observe(compute_result, names='value')

# Display widgets and output
display(widgets.VBox([principal, rate1, rate2, tax_rate, output]))

# Initial calculation
compute_result()

***

Are there other tax schemes that would work better, and not discourage intermediate liquidation?

Indeed there are schemes which are neutral to liquidation frequency. Mathematically, what we want is a tax function which yields an equivalent result when applied either after both increases or just after the second increase. This can be discovered by mathematically modeling that property and finding tax schemes which satisfy it.  

To start, consider a function $f(x)$ which describes the effective rate of return after taxes are applied to an $x$-fold increase. For example the typical flat rate approach would take away $T (x -1)$ from $x$, where $T$ is the tax rate. So the tax function $f(x)$ would be 

$$
f^*(x) = x - T (x - 1)
$$

The two scenarios we are considering are the intermediate liquidiation scenario, and the single-liquidation scenario. If an investor yields a rate of return of $x$ and then sells, they end up with $f(x)$. Applying that as principal and then achieving a rate increase of $y$, they would have $f(x)f(y)$. If they did not sell in the middle, they would instead of $f(x)f(y)$. 

The scheme is neutral if the two strategies yield the same result. That is, 

$$
f(xy) = f(x)f(y) 
$$

This is the multiplicative property. It is clear that $f^*$ does not satisfy this upon immediate inspection; the right side of the question has a $T^2$ term which the left side does not. 

The main group of functions which do observe the multiplicative property is the group of power functions, $X^C$. This inspires a tax function of 

$$
f'(x) = x^r
$$

Where $r$ is some number between $0$ and $1$ to ensure that the result is bounded by $1$ and $x$. 

In terms of the amount of tax actually collected, for $f'(x)$ it must be $x - x^r$.

Say $r$ is set to 85%. Then, an investor who triples their money would pay $3 - 3^{0.85}$ or approximately 46% of their original principal.

Below is a re-done version of the widgets above, using this tax scheme. It shows that under this scheme, the intermediate liquidation does not change the end result.

In [35]:

# Create widgets for the inputs
principal = widgets.FloatText(
    value=100
    ,description='Principal:'
    ,step=1
    ,style={'description_width': '200px'}
  
)

rate1 = widgets.FloatText(
    value=100
    ,description='First rate of increase (%):'
    ,step=1
    ,style={'description_width': '200px'}
 


)

rate2 = widgets.FloatText(
    value=100
    ,description='Second rate of increase (%):'
    ,step=1
    ,style={'description_width': '200px'}

)

tax_rate = widgets.FloatSlider(
    value=0.85,
    min=0.0,
    max=1.0,
    step=0.01,
    description='Power law factor $r$:',
    readout_format='.0%'
)

output = widgets.Output()

def compute_result(change=None):
    with output:
        output.clear_output()
        r1 = rate1.value/100 + 1
        r2 = rate2.value/100 + 1

        if r1 <0 or r2 < 0:
            print('Losses are a different thing than gains. For this widget, keep the rate of increases positive')
        else:
            # Placeholder computation
            one_liquidation = (r1*r2)**tax_rate.value
            
            intmd = r1**tax_rate.value
            two_liquidations = r2**tax_rate.value * intmd
            
            print(f"Intermediate value after liquidation : {principal.value * intmd:.2f}")
            print(f"Ending total, after second liquidation : {principal.value * two_liquidations:.2f}")
            print("")
            print(f"Ending total, one liquidation without intermediate: {principal.value * one_liquidation:.2f}")

# Link widget changes to update function
principal.observe(compute_result, names='value')
rate1.observe(compute_result, names='value')
rate2.observe(compute_result, names='value')
tax_rate.observe(compute_result, names='value')

# Display widgets and output
display(widgets.VBox([principal, rate1, rate2, tax_rate, output]))

# Initial calculation
compute_result()